In [ ]:
import pennylane as qml
from pennylane import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# 양자 장치 설정 (2 qubits)
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

# 양자회로 정의
@qml.qnode(dev, interface="torch")
def quantum_circuit(inputs, weights):
    # 입력 임베딩
    for i in range(n_qubits):
        qml.RX(inputs[i], wires=i)

    # 파라미터화된 양자회로 (Entangling + Rotations)
    qml.CNOT(wires=[0,1])
    for i in range(n_qubits):
        qml.RY(weights[i], wires=i)

    # 측정
    return [qml.expval(qml.PauliZ(i)) for i in range(n_qubits)]

# Hybrid 모델 구성
class QNLPModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.q_params = nn.Parameter(torch.randn(n_qubits))
        self.fc = nn.Linear(n_qubits, 2)  # 2-class classification

    def forward(self, x):
        # 양자 서브네트워크
        q_out = torch.stack([quantum_circuit(x[i], self.q_params) for i in range(len(x))])
        # 고전 선형 레이어
        return self.fc(q_out)

# 데이터셋 (간단한 toy data)
X = torch.tensor([[0.1, 0.2],
                  [0.4, 0.6],
                  [2.4, 2.1],
                  [3.1, 3.3]], requires_grad=False)
y = torch.tensor([0, 0, 1, 1])

# 모델 초기화
model = QNLPModel()
optimizer = optim.Adam(model.parameters(), lr=0.01)
loss_fn = nn.CrossEntropyLoss()

# 학습
epochs = 30
for epoch in range(epochs):
    optimizer.zero_grad()
    output = model(X)
    loss = loss_fn(output, y)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 5 == 0:
        pred = torch.argmax(output, dim=1)
        acc = (pred == y).float().mean()
        print(f"Epoch {epoch+1}: Loss={loss.item():.4f}, Accuracy={acc.item():.4f}")

# 테스트
print("\nFinal predictions:", torch.argmax(model(X), dim=1))